In [2]:
include("pympc.jl")

Test Summary:    | Pass  Total
test sensitivity |  267    267


PyMPC

In [34]:
using JuMP, Gurobi
using DrakeVisualizer, GeometryTypes, CoordinateTransformations
DrakeVisualizer.any_open_windows() || DrakeVisualizer.new_window()

true

In [3]:
struct StateIndex <: Integer
    name::Symbol
end

const states = StateIndex.([
:q_com_x
:q_com_y
:q_r_x
:q_r_y
:q_l_x
:q_l_y
:q_cop_x
:q_cop_y
:qd_com_x
:qd_com_y
:qd_r_x
:qd_r_y
:qd_l_x
:qd_l_y
])

for s in states
    @eval $(s.name) = $s
end

Base.to_index(s::StateIndex) = findfirst(states, s)

In [40]:
mat(x::AbstractArray) = reshape(x, length(x), 1)

mat (generic function with 2 methods)

In [5]:
struct InputIndex <: Integer
    name::Symbol
end

const inputs = InputIndex.([
    :qdd_r_x,
    :qdd_r_y,
    :qdd_l_x,
    :qdd_l_y,
    :qd_cop_x,
    :qd_cop_y
])

for s in inputs
    @eval $(s.name) = $s
end

Base.to_index(s::InputIndex) = findfirst(inputs, s)

In [6]:
# ICP dynamics
# rdd = omega^2 (r - r_cop)

g = 9.8
h = 1.0
ω = √(g / h)
Δt = 0.1

A = zeros(length(states), length(states))

for (q, qd) in zip([q_com_x, q_com_y, q_r_x, q_r_y, q_l_x, q_l_y], [qd_com_x, qd_com_y, qd_r_x, qd_r_y, qd_l_x, qd_l_y])
    A[q, qd] = 1
end
A[qd_com_x, q_com_x] = ω^2
A[qd_com_y, q_com_y] = ω^2
A[qd_com_x, q_cop_x] = -ω^2
A[qd_com_y, q_cop_y] = -ω^2

B = zeros(length(states), length(inputs))
B[q_cop_x, qd_cop_x] = 1.0
B[q_cop_y, qd_cop_y] = 1.0
B[qd_r_x, qdd_r_x] = 1.0
B[qd_r_y, qdd_r_y] = 1.0
B[qd_l_x, qdd_l_x] = 1.0
B[qd_l_y, qdd_l_y] = 1.0



1.0

In [184]:
B

14×6 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0

In [207]:
# state constraint regions

# bounds
x_min = fill(-Inf, length(states))
x_min[q_com_x] = -10
x_min[q_com_y] = -10
x_min[q_r_x] = -10
x_min[q_r_y] = -10
x_min[q_l_x] = -10
x_min[q_l_y] = -10
x_min[q_cop_x] = -10
x_min[q_cop_y] = -10
x_min[qd_com_x] = -10
x_min[qd_com_y] = -10
x_min[qd_r_x] = -10
x_min[qd_r_y] = -10
x_min[qd_l_x] = -10
x_min[qd_l_y] = -10
x_max = -x_min
X_bounds = PyMPC.geometry.Polytope[:from_bounds](mat(x_min), mat(x_max))[:assemble]()


outward_step = 0.3
inward_step = 0.1
forward_step = 0.4
backward_step = 0.4

# common state constraints
# -0.3 <= (r - c) <= -0.1
# -backward <= r - c <= forward
# 0.1 <= l - c <= 0.3
A_c = zeros(8, length(states))
b_c = zeros(8)
A_c[1, q_r_x] = 1
A_c[1, q_com_x] = -1
b_c[1] = forward_step

A_c[2, q_r_x] = -1
A_c[2, q_com_x] = 1
b_c[2] = backward_step

A_c[3, q_l_x] = 1
A_c[3, q_com_x] = -1
b_c[3] = forward_step

A_c[4, q_l_x] = -1
A_c[4, q_com_x] = 1
b_c[4] = backward_step

A_c[5, q_r_y] = 1
A_c[5, q_com_y] = -1
b_c[5] = -inward_step

A_c[6, q_r_y] = -1
A_c[6, q_com_y] = 1
b_c[6] = outward_step

A_c[7, q_l_y] = 1
A_c[7, q_com_y] = -1
b_c[7] = outward_step

A_c[8, q_l_y] = -1
A_c[8, q_com_y] = 1
b_c[8] = -inward_step

# right foot support
A_r = zeros(8, length(states))
A_r[1, qd_r_x] = 1.0
A_r[2, qd_r_x] = -1.0
A_r[3, qd_r_y] = 1.0
A_r[4, qd_r_y] = -1.0
A_r[5, q_cop_x] = 1.0
A_r[5, q_r_x] = -1.0
A_r[6, q_cop_x] = -1.0
A_r[6, q_r_x] = 1.0
A_r[7, q_cop_y] = 1.0
A_r[7, q_r_y] = -1.0
A_r[8, q_cop_y] = -1.0
A_r[8, q_r_y] = 1.0

b_r = [0.01, 0.01, 0.01, 0.01, 0.05, 0.05, 0.05, 0.05]

# left foot support
A_l = zeros(8, length(states))
A_l[1, qd_l_x] = 1.0
A_l[2, qd_l_x] = -1.0
A_l[3, qd_l_y] = 1.0
A_l[4, qd_l_y] = -1.0
A_l[5, q_cop_x] = 1.0
A_l[5, q_l_x] = -1.0
A_l[6, q_cop_x] = -1.0
A_l[6, q_l_x] = 1.0
A_l[7, q_cop_y] = 1.0
A_l[7, q_l_y] = -1.0
A_l[8, q_cop_y] = -1.0
A_l[8, q_l_y] = 1.0

b_l = [0.01, 0.01, 0.01, 0.01, 0.05, 0.05, 0.05, 0.05]

X = [PyMPC.geometry.Polytope(vcat(X_bounds[:A], A_c, A_r), mat(vcat(vec(X_bounds[:b]), b_c, b_r)))[:assemble](),
     PyMPC.geometry.Polytope(vcat(X_bounds[:A], A_c, A_l), mat(vcat(vec(X_bounds[:b]), b_c, b_l)))[:assemble]()]

# Input constraint regions

max_foot_accel = 20

u_min = fill(-Inf, length(inputs))
u_min[qd_cop_x] = -1000
u_min[qd_cop_y] = -1000
u_min[qdd_r_x] = -max_foot_accel
u_min[qdd_r_y] = -max_foot_accel
u_min[qdd_l_x] = -max_foot_accel
u_min[qdd_l_y] = -max_foot_accel
u_max = -u_min
U_bounds = PyMPC.geometry.Polytope[:from_bounds](mat(u_min), mat(u_max))[:assemble]()

A_u_r = zeros(4, length(inputs))
b_u_r = zeros(4)
A_u_r[1, qdd_r_x] = 1
b_u_r[1] = 0.01
A_u_r[2, qdd_r_x] = -1
b_u_r[2] = 0.01
A_u_r[3, qdd_r_y] = 1
b_u_r[3] = 0.01
A_u_r[4, qdd_r_y] = -1
b_u_r[4] = 0.01

A_u_l = zeros(4, length(inputs))
b_u_l = zeros(4)
A_u_l[1, qdd_l_x] = 1
b_u_l[1] = 0.01
A_u_l[2, qdd_l_x] = -1
b_u_l[2] = 0.01
A_u_l[3, qdd_l_y] = 1
b_u_l[3] = 0.01
A_u_l[4, qdd_l_y] = -1
b_u_l[4] = 0.01

X = [PyMPC.geometry.Polytope(vcat(X_bounds[:A], A_c, A_r), mat(vcat(vec(X_bounds[:b]), b_c, b_r)))[:assemble](),
     PyMPC.geometry.Polytope(vcat(X_bounds[:A], A_c, A_l), mat(vcat(vec(X_bounds[:b]), b_c, b_l)))[:assemble]()]

U = [PyMPC.geometry.Polytope(vcat(U_bounds[:A], A_u_r), vcat(U_bounds[:b], mat(b_u_r)))[:assemble](),
     PyMPC.geometry.Polytope(vcat(U_bounds[:A], A_u_l), vcat(U_bounds[:b], mat(b_u_l)))[:assemble]()]

2-element Array{PyCall.PyObject,1}:
 PyObject <pympc.geometry.Polytope instance at 0x12bc8b368>
 PyObject <pympc.geometry.Polytope instance at 0x12ba2d998>

In [208]:
c = zeros(length(states), 1)

sys = PyMPC.dynamical_systems.DTPWASystem(
    [PyMPC.dynamical_systems.DTAffineSystem[:from_continuous](A, B, c, Δt),
     PyMPC.dynamical_systems.DTAffineSystem[:from_continuous](A, B, c, Δt)],
    X,
    U)

PyObject <pympc.dynamical_systems.DTPWASystem object at 0x132b3c810>

In [209]:
N = 10
Q = 0.001 * eye(length(states))
Q[qd_com_x, qd_com_x] = 100
Q[qd_com_y, qd_com_y] = 100

R = 0.001 * eye(length(inputs))
R[qdd_r_x, qdd_r_x] = 0.1
R[qdd_r_y, qdd_r_y] = 0.1
R[qdd_l_x, qdd_l_x] = 0.1
R[qdd_l_y, qdd_l_y] = 0.1

P = Q

X_N = PyMPC.geometry.Polytope(vcat(X_bounds[:A], A_c), mat(vcat(vec(X_bounds[:b]), b_c)))[:assemble]()

controller = PyMPC.control.MPCHybridController(
sys,
N,
"two",
Q,
R,
P,
X_N)

PyObject <pympc.control.MPCHybridController instance at 0x12ba1a7e8>

In [210]:
function run_mpc(controller, x0)
    u_feedforward, x_trajectory, cost, switching_sequence = controller[:feedforward](mat(x0))
    condensed = controller[:condense_program](switching_sequence)
    u, cost = condensed[:solve](mat(x0))
    active_set = condensed[:get_active_set](mat(x0), u)
    u_offset, u_linear = condensed[:get_u_sensitivity](active_set)
#     @show maximum(abs.(u_offset + u_linear * x0 - u))
#     @assert maximum(abs.(u_offset + u_linear * x0 - u)) <= 0.01
    u_feedforward[1], u_linear
end

run_mpc (generic function with 1 method)

In [211]:
vis = Visualizer()[:pendulum]
delete!(vis)
setgeometry!(vis[:com], HyperSphere(Point(0., 0, 0), 0.1))
setgeometry!(vis[:cop], HyperSphere(Point(0., 0, 0), 0.02))

Visualizer with path prefix Symbol[:pendulum, :cop] using LCM LCMCore.LCM(Ptr{Void} @0x00007fb6d8d47360, "", RawFD(102), LCMCore.Subscription[LCMCore.Subscription{LCMCore.SubscriptionOptions{DrakeVisualizer.Comms.CommsT,DrakeVisualizer.#handle_msg#9{DrakeVisualizer.CoreVisualizer}}}(LCMCore.SubscriptionOptions{DrakeVisualizer.Comms.CommsT,DrakeVisualizer.#handle_msg#9{DrakeVisualizer.CoreVisualizer}}(DrakeVisualizer.Comms.CommsT, DrakeVisualizer.handle_msg), Ptr{Void} @0x00007fb6d2f00570)])

In [212]:
function draw_state(vis, x)
    settransform!(vis[:com], Translation(x[q_com_x], x[q_com_y], h))
    settransform!(vis[:cop], Translation(x[q_cop_x], x[q_cop_y], 0))
    setgeometry!(vis[:feet][:r], PolyLine([Point(x[q_r_x], x[q_r_y], 0), Point(x[q_com_x], x[q_com_y], h)], 0.01, false, nothing, nothing))
    setgeometry!(vis[:feet][:l], PolyLine([Point(x[q_l_x], x[q_l_y], 0), Point(x[q_com_x], x[q_com_y], h)], 0.01, false, nothing, nothing))
end

draw_state (generic function with 1 method)

In [226]:
x0 = zeros(length(states))
x0[q_com_x] = 0
x0[q_com_y] = 0
x0[q_r_x] = 0
x0[q_r_y] = -0.15
x0[q_l_x] = 0
x0[q_l_y] = 0.15
x0[q_cop_x] = 0
x0[q_cop_y] = -0.15
x0[qd_com_x] = 0.3
x0[qd_com_y] = 0.3
x0[qd_r_x] = 0
x0[qd_r_y] = 0
x0[qd_l_x] = 0
x0[qd_l_y] = 0

@assert X[1][:applies_to](mat(x0))
@assert !(X[2][:applies_to](mat(x0)))

In [232]:
u_feedforward, x_trajectory, cost, switching_sequence = controller[:feedforward](mat(x0))

for x in x_trajectory
    draw_state(vis, x)
    sleep(0.1)
end

In [243]:
function clip_cop!(x)
    q_r = [x[q_r_x], x[q_r_y]]
    q_l = [x[q_l_x], x[q_l_y]]
    q_cop = [x[q_cop_x], x[q_cop_y]]
    
    if norm(q_cop - q_r) < norm(q_cop - q_l)
        q_target = q_r
    else
        q_target = q_l
    end
    
    m = Model(solver=GurobiSolver(OutputFlag=0))
    @variable m cop[1:2]
    @objective m Min sum((cop - q_cop).^2)
    @constraints m begin
        cop .- q_target .<= 0.05
        q_target .- cop .<= 0.05
    end
        
    solve(m)
    x[q_cop_x] = getvalue(cop[1])
    x[q_cop_y] = getvalue(cop[2])
end

clip_cop! (generic function with 1 method)

In [244]:
function simulate(x0, control, ts::StepRangeLen=0:0.01:5)
    x = copy(x0)
    for t in ts
        clip_cop!(x)
        u = control(x)
        u = clamp.(u, u_min, u_max)
        ẋ = A * x + B * u
        x .+= step(ts) .* ẋ
        draw_state(vis, x)
        sleep(step(ts))
    end
end

simulate (generic function with 2 methods)

In [245]:
simulate(x0, x -> run_mpc(controller, x)[1][1:length(inputs)], 0:0.1:1)

x = [0.0, 0.0, 0.0, -0.15, 0.0, 0.15, 0.0, -0.15, 0.3, 0.3, 0.0, 0.0, 0.0, 0.0]
x = [0.0, 0.0, 0.0, -0.15, 0.0, 0.15, -0.0, -0.15, 0.3, 0.3, 0.0, 0.0, 0.0, 0.0]
u = [0.01, 0.01, 9.83458, 12.5744, 0.5, 0.5]
x = [0.03, 0.03, 0.0, -0.15, 0.0, 0.15, 0.05, -0.1, 0.3, 0.447, 0.001, 0.001, 0.983458, 1.25744]
x = [0.03, 0.03, 0.0, -0.15, 0.0, 0.15, 0.05, -0.1, 0.3, 0.447, 0.001, 0.001, 0.983458, 1.25744]
u = [0.01, 0.01, -9.73458, -12.4744, 0.983458, 4.25744]
x = [0.06, 0.0747, 0.0001, -0.1499, 0.0983458, 0.275744, 0.148346, 0.325744, 0.2804, 0.5744, 0.002, 0.002, 0.01, 0.01]
x = [0.06, 0.0747, 0.0001, -0.1499, 0.0983458, 0.275744, 0.148346, 0.325744, 0.2804, 0.5744, 0.002, 0.002, 0.01, 0.01]
u = [1.78265, 2.54691, 0.0, 0.0, 0.01, 0.01]
x = [0.08804, 0.13214, 0.0003, -0.1497, 0.0993458, 0.276744, 0.149346, 0.326744, 0.193821, 0.328377, 0.180265, 0.256691, 0.01, 0.01]
x = [0.08804, 0.13214, 0.0003, -0.1497, 0.0993458, 0.276744, 0.149346, 0.326744, 0.193821, 0.328377, 0.180265, 0.256691, 0.01, 0